<a href="https://colab.research.google.com/github/saiganeshparasa/NLP/blob/main/SA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import datetime
file = eval(open("output-LG-allratings1.txt", 'r',encoding="utf8").read())

In [4]:
df = pd.DataFrame(file)

In [5]:
df.value_counts()

reviewId                                                                                   userName           userImage                                                                                                                content                                                                                                                                                                                                                                                                                                                                                                                                                                                                               score  thumbsUpCount  reviewCreatedVersion  at                   replyContent                                                                                                                                                                                                                              

In [6]:

model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
label= []

In [8]:
for row in df['score']:
        if row < 2.0 :    label.append('0')
        elif row > 3.0: label.append('1')

        
        else:           label.append('2')

In [9]:
df['label'] = label

In [10]:
df['label'].value_counts()

1    8347
0    6737
2    2540
Name: label, dtype: int64

In [11]:
pos = df[df['label'] == 1]

In [12]:
batch_1 = df

In [13]:
tokenized = df['content'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

Token indices sequence length is longer than the specified maximum sequence length for this model (562 > 512). Running this sequence through the model will result in indexing errors


In [14]:
max_len = 512
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [15]:
np.array(padded).shape

(17624, 562)

In [16]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(17624, 562)

In [ ]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [26]:
features = last_hidden_states[0][:,0,:].numpy()

In [43]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

ValueError: ignored

In [33]:
for col in df.columns:
    print(col)

reviewId
userName
userImage
content
score
thumbsUpCount
reviewCreatedVersion
at
replyContent
repliedAt
label


In [42]:
labels=df['label']

In [41]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

NameError: ignored

In [ ]:
lr_clf.score(test_features, test_labels)

In [ ]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))